In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import pickle
from random import randint
import sys
from tqdm import tqdm_notebook as tqdm
import cv2

In [ ]:
from util.data_utils import get_SALICON_datasets
# from util.data_utils import get_raw_SALICON_datasets

train_data, val_data, test_data, mean_image = get_SALICON_datasets('Dataset/Transformed') # 128x96
#train_data, val_data, test_data, mean_image = get_raw_SALICON_datasets(dataset_folder='/tmp/pbqk24_tmp') # 640x480


In [ ]:
def NSS_loss(x, y):
        """
        Computes the Normalized Scanpath Saliency between x (output of a model)
        and y (label). X and Y are assumed to be torch tensors.
        """
        # Normalize x
        x = (x - x.mean())/x.std()
        # Create a binary mask to select values from x where the corresponding y value is > 0
        mask = y > 0 #Use threshold=0 always
        scanpath = torch.masked_select(x, mask)
        # return negative mean, as loss is minimized in training
        return -scanpath.mean()

def NSS_loss_alt(x, y):
        """
        Computes the Normalized Scanpath Saliency between x (output of a model)
        and y (label). x and y are assumed to be torch tensors.
        """
        # Normalize x
        x = (x - x.mean())/x.std()
        
        # Compute the element-wise multiplication of x and y
        scanpath = x * y
        
        # Compute the sum of the scanpath divided by the sum of values in y as NSS
        nss = scanpath.sum()/y.sum()
        
        # NSS loss = -NSS
        return -nss

def PCCLoss_torch(x, y):
    """Computes Pearson Cross Correlation loss
    :param x: prediction
    :param y: label
    """
    vx = x - torch.mean(x)
    vy = y - torch.mean(y)
    
    cc = torch.sum(vx*y) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))
    
    # since cc is in [-1, 1], and 0 is 'bad' and close to -1 or 1 is 'good', return the abs value of cc
    cc = abs(cc)
    # actually return 1 -  cc, as we need to return a loss (since 1 is good, we return loss as 1 - cc)
    loss = 1 - cc
    
    return loss

In [ ]:
from util.data_utils import OverfitSampler
from models.DSCLRCN_PyTorch import DSCLRCN #DSCLRCN_PyTorch, DSCLRCN_PyTorch2 or DSCLRCN_PyTorch3
from util.solver import Solver

batchsize = 10 # Recommended: 20
epoch_number = 10 # Recommended: 10 (epoch_number =~ batchsize/2)
net_type = 'Seg' # 'Seg' or 'CNN' Recommended: Seg
optim_str = 'SGD' # 'SGD' or 'Adam' Recommended: Adam
optim_args = {'lr': 1e-2} # 1e-2 if SGD, 1e-4 if Adam
loss_func = NSS_loss_alt # NSS_loss or torch.nn.KLDivLoss() Recommended: NSS_loss

optim = torch.optim.SGD if optim_str == 'SGD' else torch.optim.Adam

#num_train = 100
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batchsize, shuffle=True, num_workers=4, pin_memory=True)#,
                                           #sampler=OverfitSampler(num_train))
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batchsize, shuffle=True, num_workers=4, pin_memory=True)

# Attempt to train a model using the original image sizes
model = DSCLRCN(input_dim=(96, 128), local_feats_net=net_type)
# Set solver as torch.optim.SGD and lr as 1e-2, or torch.optim.Adam and lr 1e-4
solver = Solver(optim=optim, optim_args=optim_args, loss_func=loss_func)
solver.train(model, train_loader, val_loader, num_epochs=epoch_number, log_nth=50, filename_args={
    'batchsize' : batchsize,'epoch_number' : epoch_number,
    'net_type' : net_type, 'optim' : optim_str}
)

In [ ]:
#Saving the model:
model.save('pretrained/model_{}_{}_lr4_batch{}_epoch{}_model1'.format(net_type, optim_str, batchsize, epoch_number))
with open('pretrained/solver_{}_{}_lr4_batch{}_epoch{}_model1.pkl'.format(net_type, optim_str, batchsize, epoch_number), 'wb') as outf:
    pickle.dump(solver, outf, pickle.HIGHEST_PROTOCOL)

In [ ]:
# Plotting training and validation loss over iterations:

plt.subplot(2,1,1)
plt.plot(solver.train_loss_history, 'o')
plt.title('Train Loss')
plt.subplot(2,1,2)
plt.plot(solver.val_loss_history, '-o')
plt.title('Val Loss')
plt.show()

In [ ]:
# Loading a model from the saved state that produced the lowest validation loss during training:

from models.DSCLRCN_PyTorch import DSCLRCN # Requires the model class be loaded
from models.DSCLRCN_PyTorch2 import DSCLRCN as DSCLRCN2 # The second alternative model class

# Assumes the model uses models.DSCLRCN_PyTorch2 architecture. If not, this method will fail
def load_model_from_checkpoint(model_name):
    filename = "trained_models/" + model_name + ".pth"
    if torch.cuda.is_available():
        checkpoint = torch.load(filename)
    else:
        # Load GPU model on CPU
        checkpoint = torch.load(filename, map_location='cpu')
    start_epoch = checkpoint['epoch']
    best_accuracy = checkpoint['best_accuracy']
    # Find which local_feats_net the model used
    if model_name.find('Seg') > -1:
        model = DSCLRCN(input_dim=(96, 128), local_feats_net='Seg')
    else:
        model = DSCLRCN(input_dim=(96, 128), local_feats_net='CNN')
    try:
        model_class = 1
        model.load_state_dict(checkpoint['state_dict'])
    except RuntimeError:
        # The model was trained using the alternative model class
        if model_name.find('Seg') > -1:
            model = DSCLRCN2(input_dim=(96, 128), local_feats_net='Seg')
        else:
            model = DSCLRCN2(input_dim=(96, 128), local_feats_net='CNN')
        model_class = 2
        model.load_state_dict(checkpoint['state_dict'])
            
    print("=> loaded model architecture {} checkpoint '{}' (trained for {} epochs)".format(model_class, model_name, checkpoint['epoch']))

    if torch.cuda.is_available():
        model = model.cuda()
    return model

def load_model(model_name):
    model = torch.load("trained_models/" + model_name, map_location='cpu')
    print("=> loaded model_1 '{}'".format(model_name))
    
    if torch.cuda.is_available():
        model = model.cuda()
    return model

In [ ]:
#Loading some pretrained models to test them on the images:
models = []
model_names = []

# model_0: The best performing checkpoint of the model trained during this execution cycle
# model_names.append('best_model_{}_{}_lr4_batch{}_epoch{}'.format(net_type, optim_str, batchsize, epoch_number))
model_names.append('b20 e10 KLDiv Adam (best vanilla)/model_Seg_100_lr4_batch20_epoch10')

# model_1: Best model thusfar
# Load the batchsize = 20, epoch = 10 model - best produced so far
model_names.append("Model 1 NSS SGD (best faithful)/best_model_Seg_SGD_lr1_batch10_epoch10")

# model_2: Best contender for model_0
# Uses SGD and Pearson's CC loss function (proper, loss taken as 1 - abs(cc))
# model_names.append("b20 e20 PCC abs/best_model_Seg_SGD_lr4_batch20_epoch10")
# model_names.append('DSCLRCN Model 1/best_model_Seg_SGD_lr4_batch10_epoch10')


# other models


max_name_len = max([len(name) for name in model_names])
# Load the models specified above
for i, name in enumerate(model_names):
    if "best_model" in name:
        models.append(load_model_from_checkpoint(name))
    else:
        models.append(load_model(name))

print()
print("Loaded all specified models")

In [ ]:
# Loading the original images from the test set ('test_datadict.pickle': dictionary of images and fixation maps)
# NOTE: This does NOT contain any fixation maps, as these are not provided with SALICON test images
with open('Dataset/Transformed/test_datadict.pickle', 'rb') as f:
        test_data_original = pickle.load(f)
        print("Test data loaded")

# Loading the original images from the validation set ('val_datadict.pickle': dictionary of images and fixation maps)
with open('Dataset/Transformed/val_datadict.pickle', 'rb') as f:
        val_data_original = pickle.load(f)
        print("Validation data loaded")

In [ ]:
# Testing the different models on a random image from the val set:

# Pick a random test image and validation image
test_image_id = randint(0, len(test_data_original['images'])-1)
val_image_id  = randint(0, len(val_data_original['images'])-1)

# Load the images
x,y = test_data.__getitem__(test_image_id)
x_val, y_val = val_data.__getitem__(val_image_id)

# Get the original (before pre-processing) images to be displayed
original = test_data_original['images'][test_image_id]
original_val = val_data_original['images'][val_image_id]

# Loading an individual/specific image stored in /Dataset
# x = cv2.imread('Dataset/test.jpg').astype(np.float32)/255.; y = cv2.imread('Dataset/test_GT.jpg').astype(np.float32)/255. # Load the image and GT
# x = cv2.cvtColor(x,cv2.COLOR_BGR2RGB); y = cv2.cvtColor(y, cv2.COLOR_BGR2RGB) # convert from BGR (cv2) to RGB (matplotlib)
# x = cv2.resize(x,(128, 96), interpolation = cv2.INTER_AREA); y = cv2.resize(y,(128, 96), interpolation = cv2.INTER_AREA) # Resize images to 128, 96
# original = x.copy() # copy the image to use as output

# x -= mean_image # normalize x
# x = x.transpose(2,0,1) # Convert from H, W, C to C, H, W (ordering used by PyTorch tensors)
# x = torch.from_numpy(x) # Convert np arrays to tensors
# y = torch.from_numpy(y)


# Create copies of the images to pass through each model
x = x.contiguous().view(1, *x.size())
x_2 = x[:]
x_val = x_val.contiguous().view(1, *x_val.size())
x_2_val = x_val[:]
if torch.cuda.is_available():
    x = x.cuda()
    x_val = x_val.cuda()
    x_2 = x_2.cuda()
    x_2_val = x_2_val.cuda()
y = y.numpy()
y_val = y_val.numpy()



##### First model #####

x_sal = models[0](Variable(x))
if torch.cuda.is_available():
    x_sal = x_sal.cpu()
x_sal_nmp = x_sal.squeeze().data.numpy()


# Sigma used by both/all models as all inputs same shape
sigma = 0.035*min(x_sal_nmp.shape) # Define a sigma to be used for Gaussian blurring

# Blur the saliency map

# Sigma value used by all models, as all inputs are of same shape
sigma = 0.035*min(x_sal_nmp.shape) # Define a sigma to be used for Gaussian blurring

x_sal_nmp = cv2.GaussianBlur(x_sal_nmp, (int(4*sigma), int(4*sigma)), sigma)

x_val_sal = models[0](Variable(x_val))
if torch.cuda.is_available():
    x_val_sal = x_val_sal.cpu()
x_val_sal_nmp = x_val_sal.squeeze().data.numpy()
# Blur the saliency map
x_val_sal_nmp = cv2.GaussianBlur(x_val_sal_nmp, (int(4*sigma), int(4*sigma)), sigma)

##### Second model #####
x_2_sal = models[1](Variable(x_2))
if torch.cuda.is_available():
    x_2_sal = x_2_sal.cpu()
x_2_sal_nmp = x_2_sal.squeeze().data.numpy()
# Blur the saliency map
x_2_sal_nmp = cv2.GaussianBlur(x_2_sal_nmp, (int(4*sigma), int(4*sigma)), sigma)


x_2_val_sal = models[1](Variable(x_2_val))
if torch.cuda.is_available():
    x_2_val_sal = x_2_val_sal.cpu()
x_2_val_sal_nmp = x_2_val_sal.squeeze().data.numpy()
# Blur the saliency map
x_2_val_sal_nmp = cv2.GaussianBlur(x_2_val_sal_nmp, (int(4*sigma), int(4*sigma)), sigma)


# Plot the output
plt.figure(figsize=(20,8))

##### Testing set image #####
plt.subplot(2,4,1); plt.title('Original')
plt.imshow(original, vmin=0, vmax=1)
plt.subplot(2,4,2); plt.title('Ground Truth')
plt.imshow(y, cmap='gray', vmin=0, vmax=1)

# First model
plt.subplot(2,4,3)
plt.imshow(x_sal_nmp, cmap='gray', vmin=0, vmax=1); plt.title(model_names[0])
# Second model
plt.subplot(2,4,4)
plt.imshow(x_2_sal_nmp, cmap='gray', vmin=0, vmax=1); plt.title(model_names[1])

##### Validation set image #####
plt.subplot(2,4,5); plt.title('Original Val')
plt.imshow(original_val, vmin=0, vmax=1)
plt.subplot(2,4,6); plt.title('Ground Truth Val')
plt.imshow(y_val, cmap='gray', vmin=0, vmax=1)

# First model
plt.subplot(2,4,7)
plt.imshow(x_val_sal_nmp, cmap='gray'); plt.title(model_names[0])
# Second model
plt.subplot(2,4,8)
plt.imshow(x_2_val_sal_nmp, cmap='gray'); plt.title(model_names[1])

# plt.savefig('ResExamples/example_test_'+str(test_image_id)+'_val_'+str(val_image_id)+'.png')
plt.show()

In [ ]:
print(x_val_sal_nmp.min(), x_val_sal_nmp.max())
test = x_val_sal_nmp/x_val_sal_nmp.max()
print(test.min(), test.max())
cv2.imshow("batch20_epoch10_val", test)
cv2.waitKey(0)

In [ ]:
# Define the Pearson Cross Correlation loss function, using numpy:
def PCC_loss_numpy(x, y):
    """Computes Pearson Cross Correlation loss
    :param x: prediction
    :param y: label
    """
    vx = x - np.mean(x)
    vy = y - np.mean(y)
    
    loss = np.sum(vx*y) / (np.sqrt(np.sum(vx ** 2)) * np.sqrt(np.sum(vy ** 2)))
    
    return loss

# Define the Normalized Scanpath Saliency loss function, using numpy:
def NSS_loss_numpy(x, y):
        """
        Computes the Normalized Scanpath Saliency between x (output of a model)
        and y (label). X and Y are assumed to be torch tensors.
        """
        # Normalize x
        x = (x - x.mean())/x.std()
        # Create a binary mask to select values from x where the corresponding y value is > 0
        mask = y <= 0 #Use threshold=0 always, reversed as numpy function masks out True and keeps False
        scanpath = np.ma.masked_array(x, mask)
        # return negative mean, as loss is minimized in training
        return -scanpath.mean()
    
# Define NSS function
def NSS_numpy(x, y):
    return -NSS_loss_numpy(x, y)

In [ ]:
# Define a function for testing a model
# The pipeline is different depending on if the loss fn is numpy or torch based (check naively implemented so far)
# This is because the proper method of testing, applying a gaussian blur to the output map, has only been implemented to use numpy.
# Thus, the torch peipeline will not yield correct results as it does not correctly follow the method of the original paper.
def test_model(model, data_source, loss_fn=PCC_loss_numpy, input_size=(640, 480)):
    test_loader = torch.utils.data.DataLoader(data_source, batch_size=5, shuffle=True, num_workers=4)
    testLosses = []
    
    for data in tqdm(test_loader):
        inputs, labels = data
        if torch.cuda.is_available():
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        else:
            inputs = Variable(inputs)
            labels = Variable(labels)
            
        ### TORCH PIPELINE ###
        if loss_fn == PCCLoss_torch:
            # Produce the output
            outputs = model(inputs).squeeze()

            # Zero-center the fixation maps by dividing each value in each fixation map by the sum of all values in that
            # fixation map
            labels_sum = torch.sum(labels.contiguous().view(labels.size(0),-1), dim=1)
            labels /= labels_sum.contiguous().view(*labels_sum.size(), 1, 1).expand_as(labels)
        
        ### NUMPY PIPELINE ###
        #if loss_fn == PCCLoss_numpy:
        else:
            # Produce the output
            outputs = model(inputs).squeeze()
            # Move the output to the CPU so we can process it using numpy
            outputs = outputs.cpu().data.numpy()
            
            # Show the input and output, side to side
#             cv2.imshow("Input & Output", cv2.cvtColor(np.hstack(((inputs.cpu().data.numpy()[0].transpose(1, 2, 0) + mean_image), np.repeat(outputs[0, :, :, np.newaxis], 3, axis=2))), cv2.COLOR_RGB2BGR))
#             cv2.waitKey(0)
            
            # Resize the images to input size
            outputs = np.array([cv2.resize(output, input_size) for output in outputs])
            
            # Apply a Gaussian filter to blur the saliency maps
            sigma = 0.035*min(input_size[0], input_size[1])
            outputs = np.array([cv2.GaussianBlur(output, (int(4*sigma), int(4*sigma)), sigma) for output in outputs])

            # Zero-center the fixation maps by dividing each value in each fixation map by the sum of all values in that
            # fixation map
#             labels_sum = torch.sum(labels.contiguous().view(labels.size(0),-1), dim=1)
#             labels /= labels_sum.contiguous().view(*labels_sum.size(), 1, 1).expand_as(labels)
            
            labels = labels.cpu().numpy()
            # Show the blurred output and the GT, side to side
#             cv2.imshow("Ground Truth & Blurred Output", np.hstack((labels[0], outputs[0])))
#             cv2.waitKey(0)
        
        ### BOTH PIPELINES ###
        testLosses.append(loss_fn(outputs, labels).item())
    
    return testLosses

In [ ]:
# Obtaining PCC Loss values on the test set for different models:

# loss_fn = nn.KLDivLoss()
# Use Pearson Cross Correlation loss
loss_fn = NSS_numpy

# test on validation data as we don't have ground truths for the test data (this was also done in original DSCLRCN paper)

test_losses = []
for model in tqdm(models):
    test_losses.append(test_model(model, val_data, loss_fn=loss_fn, input_size=(128, 96)))

# Print out the result
if loss_fn == PCC_loss_numpy:
    print('Pearson Cross Correlation Loss on Validation set:')
elif loss_fn == NSS_loss_numpy:
    print('Normalized Scanpath Saliency Loss on Validation set:')
else:
    print('Normalized Scanpath Saliency on Validation set:')

for i, loss in enumerate(test_losses):
    print(('[{}] {:' + str(max_name_len) + '} : {:6f}').format(i, model_names[i], np.mean(loss)))
